# Домашняя работа №1
## Урок 1. Введение в Большие Данные

1) Ознакомьтесь / изучите библиотеки:    

— https://joblib.readthedocs.io    
— https://www.dask.org    
— https://www.ray.io    

2) Установите Docker (https://docs.docker.com/desktop)    
 
3) Посчитать средний балл фильмов. Подробнее в нотебуке.    

Файл для ДЗ во вложении к уроку.    

In [49]:
import opendatasets as od
import pandas as pd
import time
import math
from pathlib import Path
import json
from functools import reduce
from joblib import Parallel, delayed

## Скачиваем датасет
Потребуется kaggle ключ для доступа, который сгенерирован на сайте и записан в файл kaggle.json. 

In [50]:
dataset_path = Path('imdb-user-reviews', 'song_lyrics.csv')
if not dataset_path.is_file():
    od.download('https://www.kaggle.com/datasets/sadmadlad/imdb-user-reviews')

Skipping, found downloaded files in ".\imdb-user-reviews" (use force=True to force download)


# Домашняя работа

Сделайте mapper и reducer, чтобы посчитать среднее и дисперсию оценок за фильм.

Реализация через цикл (предпогаем, что мы не знаем сколько у нас фильмов):

In [51]:
%%time
n, mean, M2 = 0, 0.0, 0

for path in Path('imdb-user-reviews').glob('**/*'):
    if path.is_file() and path.suffix == '.json':
        with open(path, 'r') as f:
            info = json.load(f)
        score = float(info['movieIMDbRating'])
        n += 1
        delta = score - mean
        mean += delta / n
        M2 += delta * (score - mean)

print(mean, (M2 / n) ** (1/2))

8.03 1.0517128885774865
CPU times: total: 0 ns
Wall time: 5.98 ms


На основе этого кода соберите mapper и reducer:

In [52]:
def mapper(path):
    if path.is_file() and path.suffix == '.json':
        with open(path, 'r') as f:
            info = json.load(f)
        score = float(info['movieIMDbRating'])
        return (score, )
        


def reducer(score_data1, score_data2):
    if score_data1 is None and score_data2 is None:
        return None
    elif score_data1 is None:
        return score_data2
    elif score_data2 is None:
        return score_data1
    else:
        scores = []
        if len(score_data1) == 1:
            n, mean, M2 = 0, 0.0, 0
            scores.append(score_data1[0])
        else:
            n, mean, M2 = score_data1
        scores.append(score_data2[0])
        for score in scores:
            n += 1
            delta = score - mean
            mean += delta / n
            M2 += delta * (score - mean)
    return n, mean, M2

In [53]:
%%time
n, mean, M2 = reduce(reducer, map(mapper, Path('imdb-user-reviews').glob('**/*')))
print(mean, (M2 / n) ** (1/2))

8.03 1.0517128885774865
CPU times: total: 0 ns
Wall time: 5.98 ms


## Распараллеливаем данные:
По аналогии с методами изученными на семинаре.

In [55]:
%%time
N=2
data_path = Path('imdb-user-reviews').glob('**/*')  #создаем генератор и разбили данные на 2 части
#step 1:
mapped = Parallel(n_jobs=N)(delayed(mapper)(path) for path in data_path)
#step 2:
n, mean, M2 = reduce(reducer, mapped)
print(mean, (M2 / n) ** (1/2))


8.03 1.0517128885774865
CPU times: total: 15.6 ms
Wall time: 22.7 ms
